In [36]:
spark

In [49]:
from pyspark.sql.functions import array, coalesce, concat,  col

## Placeholder

In [37]:
import hsfs
from hops import hdfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [38]:
node_fg = fs.get_training_dataset("node_td", 1)
node_labels_df = node_fg.read().select("id","is_sar")

In [39]:
node_labels_df.show(5)

+----+------+
|  id|is_sar|
+----+------+
|7605|     1|
|6075|     1|
|5814|     1|
|9360|     1|
|4626|     1|
+----+------+
only showing top 5 rows

## Placeholder

In [40]:
node_embeddings_df = spark.read.csv("hdfs:///Projects/{}/Resources/node_embeddings_features_2_2_16.csv".format(hdfs.project_name(),inferSchema=True,header=False))

In [41]:
feature_names = node_embeddings_df.columns
feature_names = ["_" + s + "c"  for s in feature_names]
feature_names[0]= 'id'
node_embeddings_node_embeddings_df = node_embeddings_df.toDF(*feature_names)

In [42]:
feature_names.remove('id')

In [54]:
emb_td = node_embeddings_node_embeddings_df.join(node_labels_df, ['id'])

In [55]:
emb_td = emb_td.drop("id").withColumn("embedding", array(feature_names)).select("is_sar","embedding").withColumnRenamed("is_sar","target")

## Placeholder

In [61]:
non_sar_df = emb_td.where(col("target")==0)

In [62]:
sar_df = emb_td.where(col("target")==1)

In [63]:
# Now that the data has been prepared, let's split the dataset into a training and test dataframe
[non_sar_train_df, non_sar_eval_df] = non_sar_df.randomSplit([0.8, 0.02],seed = 12345)

In [ ]:
non_sar_td = fs.create_training_dataset(name="gan_non_sar_training_df",
                                       version=1,
                                       data_format="tfrecord",
                                       label=["target"], 
                                       statistics_config = None, 
                                       description="non sar dataset for gan training")
non_sar_td.save(non_sar_train_df)

## Placeholder

In [65]:
eval_df = non_sar_eval_df.union(sar_df)

In [ ]:
gan_eval_ds = fs.create_training_dataset(name="gan_eval",
                                       version=1,
                                       data_format="tfrecord",
                                       label=["target"], 
                                       statistics_config = None, 
                                       description="evaluation dataset for gan training")
gan_eval_ds.save(eval_df)